In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes --quiet

In [ ]:

# Install necessary packages (run in Colab or local virtual environment)


# 1. Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
import torch

# 2. Load your dataset (replace with your path or use `load_dataset` with a script or file)
dataset = load_dataset("json", data_files="/content/gfg_html_css_articles_concurrency.json", split="train")

# Keep only relevant fields
dataset = dataset.map(lambda x: {
    "text": f"{x['title']}\n<start_html>\n{x['html_code']}\n<end_html>\n<start_css>\n{x['css_code']}\n<end_css>"
})

# 3. Load tokenizer and model (using Gemma 2B as an example; substitute with a 4-bit-supported model if needed)
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Pad with eos

# Load model with 4-bit quantization
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# 4. PEFT QLoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

# 5. Tokenize data
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./qlora-html-css",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    max_steps=1000,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2
)

# 7. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 9. Train
trainer.train()

# 10. Save model
trainer.save_model("./qlora-html-css")
tokenizer.save_pretrained("./qlora-html-css")

# 11. Inference example
from transformers import pipeline
pipe = pipeline("text-generation", model="./qlora-html-css", tokenizer=tokenizer, device=0)
pipe("responsive login page with form")  # Will output HTML+CSS format


In [ ]:
# chat = [
#     {"role": "system", "content": "You are a smart html and css generator."},
#     {"role": "user", "content": "Generate a responsive login page with form"}
# ]
# pipeline = pipeline(task="text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")
response = pipe("Design a Google Chrome Page Template using HTML and CSS. Format the code properly and if you don't know reply, I don't know",max_new_tokens=1024)
print(response)

# pipe("responsive login page with form")  # Will output HTML+CSS format

In [ ]:
from transformers import pipeline, AutoTokenizer, GenerationConfig, AutoModelForCausalLM

model_path = "rajdesai1510/small_fine_tuned_gemma"  # path to your fine-tuned QLoRA model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Optional: attach generation config
generation_config = GenerationConfig(
    max_new_tokens=1024,
    temperature=0.5,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
    no_repeat_ngram_size=4,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

pipe = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=tokenizer,
    device=0,
    generation_config=generation_config
)

def format_prompt(title):
    return f"Title: {title}\nOutput:\n"

# Example usage
title = "Design a Google Chrome Page Template using HTML and CSS. Format Properly."
result = pipe(format_prompt(title))[0]['generated_text']
generated_code = result.split("Output:\n", 1)[-1].strip()
print(generated_code)


In [ ]:
from transformers import pipeline, AutoTokenizer, GenerationConfig, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

model_path = "rajdesai1510/small_fine_tuned_gemma"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit using bitsandbytes

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # Use "fp4" or "nf4"
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically allocates across devices
    trust_remote_code=True
)

# Optional: set generation config
generation_config = GenerationConfig(
    max_new_tokens=1024,
    temperature=0.5,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
    no_repeat_ngram_size=4,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Create pipeline with quantized model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config
)

def format_prompt(title):
    return f"Title: {title}\nOutput:\n"

# Example inference
title = "Design a Google Chrome Page Template using HTML and CSS. Format Properly."
result = pipe(format_prompt(title))[0]['generated_text']
generated_code = result.split("Output:\n", 1)[-1].strip()
print(generated_code)

In [ ]:
generated_code